In [1]:
import pandas as pd
import numpy as np
import math
import io

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pydotplus

from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor 

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import plot_tree
from sklearn.externals.six import StringIO

from IPython.display import SVG
from graphviz import Source
from IPython.display import display

## Edit Print options
desired_width = 500
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 600)


missing_values = ["n/a", "na", "--","NAN"," ","nan","NaN"]
data = pd.read_csv('datafile2.csv',na_values = missing_values)
test_data = pd.read_csv('test.csv',na_values = missing_values)

## Removing missing values
# data.dropna()

## Just a temporary dataset to test the capabilities of the model
temp_data = data[['brand','screen_size','touchscreen','discrete_gpu','gpu','ram','ssd', 'detachable_keyboard','weight',
                      'min_price','max_price','diff_price']]


## Filling in missing values
temp_data['detachable_keyboard'] = temp_data['detachable_keyboard'].fillna(0)
temp_data['detachable_keyboard'] = temp_data['detachable_keyboard'].astype(bool)

temp_data['weight'] = temp_data['weight'].fillna(temp_data['weight'].mean())



//anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [2]:

## Seperating categorical - binary - numerical variables
X_cat = temp_data[['brand','gpu']]
X_bin = temp_data[['touchscreen','discrete_gpu','detachable_keyboard']]
X_num = temp_data[['screen_size','ram','ssd','weight']]

## Creating dummy variables:
X_cat_dummies = pd.get_dummies(X_cat, drop_first=True)

## Merging input data
X = pd.concat([X_cat_dummies,X_bin,X_num], axis=1)
# X = pd.concat([X_cat_dummies,X_num], axis=1)
# X = X_cat_dummies

## Defining output data
Y = temp_data[['min_price','max_price']]

## Splitting the dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

print(X_train)
# Model1 = DecisionTreeRegressor(criterion="mae")
Model1 = RandomForestRegressor(n_estimators = 500,criterion='mae',random_state = 0)
Model1.fit(X_train,Y_train)


     brand_ALIENWARE  brand_APPLE  brand_ASUS  brand_DELL  brand_GOOGLE  brand_HP  brand_HUAWEI  brand_JUMPER  brand_LENOVO  brand_LG  brand_MICROSOFT  brand_MSI  brand_OTHER  brand_RAZER  brand_RCA  ...  gpu_NVIDIA GEFORCE MX250  gpu_NVIDIA GEFORCE RTX 2060  gpu_NVIDIA GEFORCE RTX 2070  gpu_NVIDIA GEFORCE RTX 2070 MAX-Q  gpu_NVIDIA GEFORCE RTX 2080  gpu_NVIDIA GEFORCE RTX 2080 MAX-Q  gpu_NVIDIA QUADRO M1000M  gpu_POWERVR GX6250  touchscreen  discrete_gpu  detachable_keyboard  screen_size  \
324                0            0           0           0             0         0             0             0             0         0                0          1            0            0          0  ...                         0                            0                            0                                  0                            0                                  0                         0                   0            0             1                False         15.6   
297   

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [3]:
## Predicting the target values
Y_pred = Model1.predict(X_test)

## Calculating the score
MAE = mean_absolute_error(Y_test, Y_pred, multioutput='raw_values')
print(MAE[0]+MAE[1])

## Turn DF to array
Y_test_array = Y_test.to_numpy()

Sum_error = 0
Y_pred_min_array = []
for i in range(len(Y_pred)):
#     print(Y_pred[i],Y_test_array[i])
    Y_pred_min = Y_pred[i][0]
    Y_pred_max = Y_pred[i][1]
    Y_test_min = Y_test_array[i][0]
    Y_test_max = Y_test_array[i][1]
    
    Y_pred_min_array.append(Y_pred_min)
    
    Error_min = abs(Y_pred_min - Y_test_min)
    Error_max = abs(Y_pred_max - Y_test_max)
    Total_error = Error_min + Error_max
    
    Sum_error = Sum_error + Total_error
    counter = i

Avg_error = Sum_error / counter
MSAE = Avg_error
print(MSAE)



355.94010414201205
358.0587952380952


In [4]:
## Printing the decision tree
# FEATURE_NAMES = X_train.columns[:]


# dot_data = export_graphviz(Model1,
#                                out_file=None,
#                                feature_names=FEATURE_NAMES,
#                                filled = True)
# graph = Source(dot_data)

# # dot_data = io.StringIO()
# # export_graphviz(Model1, out_file=dot_data, rounded=True, filled=True)
# # filename = "tree.png"
# # pydotplus.graph_from_dot_data(dot_data.getvalue()).write_png(filename)
# # plt.figure(figsize=(300,100))
# # img = mpimg.imread(filename)
# # imgplot = plt.imshow(img)
# # plt.show()

# plt.figure(figsize=(120,50))
# plot_tree(Model1, feature_names=FEATURE_NAMES,fontsize=11, rounded=True)
# plt.show()

In [5]:
# Visualising the Random Forest Regression results 
  
# arange for creating a range of values 
# from min value of x to max  
# value of x with a difference of 0.01  
# between two consecutive values 
# X_grid = np.arange(X_test, max(x), 0.01)  
# X_ram
  
# reshape for reshaping the data into a len(X_grid)*1 array,  
# i.e. to make a column out of the X_grid value                   
# X_grid = X_grid.reshape((len(X_grid), 1)) 
  
# Scatter plot for original data 
# plt.scatter(x, y, color = 'blue')   
  
# plot predicted data 
# plt.plot(Y_pred_min_array,  
#          color = 'green')  
# plt.title('Random Forest Regression') 
# plt.xlabel('Position level') 
# plt.ylabel('Max_price') 
# plt.show()